# DA for AQMS in QLD
### The current version is to wrangle and clean the AQ data from the AQMS in QLD, or AMNS in QLD

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%ls

 Volume in drive C is Windows
 Volume Serial Number is 9823-A664

 Directory of C:\Users\CHENGG\Desktop\pythonProject\DA_AQ_hub

04/11/2024  03:41 PM    <DIR>          .
04/11/2024  03:21 PM    <DIR>          ..
04/11/2024  03:41 PM                41 .gitignore
04/11/2024  03:48 PM    <DIR>          .idea
04/11/2024  03:39 PM    <DIR>          .ipynb_checkpoints
04/11/2024  03:41 PM             1,818 main_script.ipynb
               2 File(s)          1,859 bytes
               4 Dir(s)  395,780,964,352 bytes free


In [3]:
%ls C:\Users\CHENGG\Desktop\IA317900\data

 Volume in drive C is Windows
 Volume Serial Number is 9823-A664

 Directory of C:\Users\CHENGG\Desktop\IA317900\data

04/11/2024  01:43 PM    <DIR>          .
04/11/2024  01:42 PM    <DIR>          ..
04/11/2024  02:37 PM    <DIR>          BoatCreek
04/11/2024  02:37 PM    <DIR>          MomerialPark
04/11/2024  02:32 PM    <DIR>          SouthGladstone
               0 File(s)              0 bytes
               5 Dir(s)  395,781,009,408 bytes free


In [19]:
# the location of the data
data_folder = 'C:\\Users\\CHENGG\\Desktop\\IA317900\\data\\'
data_BC, data_MP, data_SG = data_folder + 'BoatCreek\\', data_folder + 'MomerialPark\\', data_folder + 'SouthGladstone\\'
file_BC = 'air-quality-data-boa-2024-10-28-to-2024-11-04.csv'
file_MP = 'air-quality-data-cgm-2024-10-28-to-2024-11-04.csv'
file_SG = 'air-quality-data-qal-2024-10-28-to-2024-11-04.csv'

In [28]:
# load the data
df_bc = pd.read_csv(data_BC+file_BC)
df_mp = pd.read_csv(data_MP+file_MP)
df_sg = pd.read_csv(data_SG+file_SG)

In [29]:
# try to wrangle the data
df_bc.head(11)

,Station,Datetime (UTC),Parameter,Measurement,Measurement units,Measurement running average,Measurement running average units,Validated
0,Boat Creek,2024-11-04T03:00:00.000Z,Nitrogen dioxide,0.000,ppm,0.000,ppm (1hr avg),N
1,Boat Creek,2024-11-04T02:00:00.000Z,Nitrogen dioxide,0.000,ppm,0.000,ppm (1hr avg),N
2,Boat Creek,2024-11-04T01:00:00.000Z,Nitrogen dioxide,0.000,ppm,0.000,ppm (1hr avg),N
3,Boat Creek,2024-11-04T00:00:00.000Z,Nitrogen dioxide,0.003,ppm,0.003,ppm (1hr avg),N
4,Boat Creek,2024-11-03T23:00:00.000Z,Nitrogen dioxide,0.002,ppm,0.002,ppm (1hr avg),N
5,Boat Creek,2024-11-03T22:00:00.000Z,Nitrogen dioxide,0.003,ppm,0.003,ppm (1hr avg),N
6,Boat Creek,2024-11-03T21:00:00.000Z,Nitrogen dioxide,0.005,ppm,0.005,ppm (1hr avg),N
7,Boat Creek,2024-11-03T20:00:00.000Z,Nitrogen dioxide,0.004,ppm,0.004,ppm (1hr avg),N
8,Boat Creek,2024-11-03T19:00:00.000Z,Nitrogen dioxide,0.002,ppm,0.002,ppm (1hr avg),N
9,Boat Creek,2024-11-03T18:00:00.000Z,Nitrogen dioxide,0.002,ppm,0.002,ppm (1hr avg),N


In [30]:
df_mp.head(11)

,Station,Datetime (UTC),Parameter,Measurement,Measurement units,Measurement running average,Measurement running average units,Validated
0,Memorial Park,2024-11-04T03:00:00.000Z,Nitrogen dioxide,0.003,ppm,0.003,ppm (1hr avg),N
1,Memorial Park,2024-11-04T02:00:00.000Z,Nitrogen dioxide,0.002,ppm,0.002,ppm (1hr avg),N
2,Memorial Park,2024-11-04T01:00:00.000Z,Nitrogen dioxide,0.002,ppm,0.002,ppm (1hr avg),N
3,Memorial Park,2024-11-04T00:00:00.000Z,Nitrogen dioxide,0.003,ppm,0.003,ppm (1hr avg),N
4,Memorial Park,2024-11-03T23:00:00.000Z,Nitrogen dioxide,0.002,ppm,0.002,ppm (1hr avg),N
5,Memorial Park,2024-11-03T22:00:00.000Z,Nitrogen dioxide,0.004,ppm,0.004,ppm (1hr avg),N
6,Memorial Park,2024-11-03T21:00:00.000Z,Nitrogen dioxide,0.010,ppm,0.010,ppm (1hr avg),N
7,Memorial Park,2024-11-03T20:00:00.000Z,Nitrogen dioxide,0.007,ppm,0.007,ppm (1hr avg),N
8,Memorial Park,2024-11-03T19:00:00.000Z,Nitrogen dioxide,0.004,ppm,0.004,ppm (1hr avg),N
9,Memorial Park,2024-11-03T18:00:00.000Z,Nitrogen dioxide,0.003,ppm,0.003,ppm (1hr avg),N


In [31]:
df_sg.head(11)

,Station,Datetime (UTC),Parameter,Measurement,Measurement units,Measurement running average,Measurement running average units,Validated
0,South Gladstone,2024-11-04T03:00:00.000Z,Nitrogen dioxide,0.002,ppm,0.002,ppm (1hr avg),N
1,South Gladstone,2024-11-04T02:00:00.000Z,Nitrogen dioxide,0.002,ppm,0.002,ppm (1hr avg),N
2,South Gladstone,2024-11-04T01:00:00.000Z,Nitrogen dioxide,0.001,ppm,0.001,ppm (1hr avg),N
3,South Gladstone,2024-11-04T00:00:00.000Z,Nitrogen dioxide,0.002,ppm,0.002,ppm (1hr avg),N
4,South Gladstone,2024-11-03T23:00:00.000Z,Nitrogen dioxide,0.003,ppm,0.003,ppm (1hr avg),N
5,South Gladstone,2024-11-03T22:00:00.000Z,Nitrogen dioxide,0.005,ppm,0.005,ppm (1hr avg),N
6,South Gladstone,2024-11-03T21:00:00.000Z,Nitrogen dioxide,0.008,ppm,0.008,ppm (1hr avg),N
7,South Gladstone,2024-11-03T20:00:00.000Z,Nitrogen dioxide,0.011,ppm,0.011,ppm (1hr avg),N
8,South Gladstone,2024-11-03T19:00:00.000Z,Nitrogen dioxide,0.003,ppm,0.003,ppm (1hr avg),N
9,South Gladstone,2024-11-03T18:00:00.000Z,Nitrogen dioxide,0.001,ppm,0.001,ppm (1hr avg),N
